In [0]:
from pyspark.sql.functions import *

In [0]:
%run /Workspace/Users/brunamiekobms@gmail.com/brunamiekobms@gmail.com_ce/sales/02_Pipeline_Utils_And_Silver_Setup

In [0]:
BRONZE_TABLE_NAME = "sales.sales_bronze"
SILVER_TABLE_NAME = "sales.sales_silver"
CHECKPOINT_PATH_SILVER = "/Volumes/workspace/sales/sales_silver/checkpoint/" 
print(f"Usando Checkpoint Location: {CHECKPOINT_PATH_SILVER}")

Usando Checkpoint Location: /Volumes/workspace/sales/sales_silver/checkpoint/


In [0]:
spark.sql(f"""
  CREATE TABLE IF NOT EXISTS {SILVER_TABLE_NAME} (
    TransactionNo STRING,       
    CustomerNo STRING,          
    Transaction_Datetime TIMESTAMP, 
    Total_Price DECIMAL(18,2),    
    ProductNo STRING,
    Product STRING,
    Price DECIMAL(18,2),
    Quantity INTEGER,
    Country STRING,
    Update_Time_Silver TIMESTAMP,
    Ingestion_Time_Bronze TIMESTAMP, 
    Raw_Data_Source STRING
    )
  USING DELTA 
""")
print(f"Tabela Silver de destino '{SILVER_TABLE_NAME}' recriada.")

Tabela Silver de destino 'sales.sales_silver' recriada.


In [0]:
pipeline = SilverSalesPipeline(
    spark=spark, 
    silver_table_name=SILVER_TABLE_NAME
)

In [0]:
print(f"\nLendo dados da Bronze em modo Streaming: {BRONZE_TABLE_NAME}")
bronze_stream_df = (
  spark.readStream
    .format("delta")
    .table(BRONZE_TABLE_NAME) 
)




Lendo dados da Bronze em modo Streaming: sales.sales_bronze


In [0]:
silver_transformed_df = pipeline.apply_transformations(bronze_stream_df)



In [0]:
print("\nIniciando Pipeline de Streaming...")
query = (
    silver_transformed_df.writeStream
        .foreachBatch(pipeline.get_upsert_function())    
        .outputMode("update")
        .trigger(availableNow=True) 
        .option("checkpointLocation", CHECKPOINT_PATH_SILVER)
        .option("partitionBy", "Update_Time_Silver")
        .start()
)

print(f"\n🚀 Pipeline de Streaming iniciado com checkpoint")


Iniciando Pipeline de Streaming...

🚀 Pipeline de Streaming iniciado com checkpoint
